## 数据预处理

In [1]:
import pandas as pd

# 读取CSV文件
left_data = pd.read_csv('left.csv')
right_data = pd.read_csv('right.csv')
up_data = pd.read_csv('up.csv')
down_data = pd.read_csv('down.csv')

# 定义一个函数来保留正向冲击
def retain_positive_spikes(data):
    return data[data > data.mean()]

# 对每个传感器数据进行处理，保留正向冲击
left_data_processed = left_data.apply(retain_positive_spikes)
right_data_processed = right_data.apply(retain_positive_spikes)
up_data_processed = up_data.apply(retain_positive_spikes)
down_data_processed = down_data.apply(retain_positive_spikes)
# 保存预处理后的数据
left_data_processed.to_csv('./data/left_processed.csv', index=False)
right_data_processed.to_csv('./data/right_processed.csv', index=False)
up_data_processed.to_csv('./data/up_processed.csv', index=False)
down_data_processed.to_csv('./data/down_processed.csv',index=False)

## 模型训练

In [ ]:
#测试Tensoflow
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
import tensorflow as tf


# 读取预处理后的CSV文件
left_data = pd.read_csv('./data/left_processed.csv')
right_data = pd.read_csv('./data/right_processed.csv')
up_data = pd.read_csv('./data/up_processed.csv')
down_data = pd.read_csv('./data/down_processed.csv')

# 添加标签列
left_data['label'] = 'left'
right_data['label'] = 'right'
up_data['label'] = 'up'
down_data['label'] = 'down'

# 合并所有数据
all_data = pd.concat([left_data, right_data, up_data, down_data])

# 分离特征和标签
X = all_data.drop(columns=['label'])
y = all_data['label']

# 将标签转换为数字编码
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# 分割训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 定义模型
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax')
])

# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

# 保存模型
model.save('/mnt/data/sensor_model.h5')
